# Imports

In [1]:
import pandas as pd
import numpy as np

---

# Carga de datos

In [2]:
auctions = pd.read_pickle("../../../../data/tp2/auctions_tp2_formateado.pkl")

In [3]:
targets_competencia = pd.read_pickle("../../../../data/tp2/ref_hashes_target.pkl")

---

### De auctions, me quedo con sólo datos que correspondan a la competencia

In [4]:
auctions_competencia = auctions.loc[auctions["device_id"].isin(targets_competencia.index)]

---

# Cargo features generados hasta el momento

## Sets de entrenamiento

In [5]:
# Ventanas de tres dias
Xs = {}
try:
    print("Cargando features")
    for ventana_nro in range(1,5):    
        Xs[ventana_nro] = pd.read_csv("../../features/ventana_{}_entrenar_auctions.csv".format(ventana_nro), index_col=0)
except FileNotFoundError:
    print("No se encontraron los features, se generarán los dataframe requeridos")
    for ventana_nro in range(1,5):    
        Xs[ventana_nro] = auctions.loc[auctions["ventana_{}".format(ventana_nro)]]["device_id"] \
            .drop_duplicates().to_frame().set_index("device_id").copy()

Cargando features


## Set que se usará para predecir

In [5]:
# Features para hacer las predicciones que se suben, corresponden a la ventana 7
Xs_predecir = {}

try:
    print("Cargando features usados para predecir")
    Xs_predecir[7] = pd.read_csv("../../features/predecir_auctions.csv", index_col=0)
except FileNotFoundError:
    print("No se encontraron los features usados para predecir, se generará el dataframe requerido")
    Xs_predecir[7] = targets_competencia.copy()

Cargando features usados para predecir
No se encontraron los features usados para predecir, se generará el dataframe requerido


## Diccionario de features

In [7]:
features = {}
# Todos los datasets tienen la misma cantidad y nombre de features, tomo la ventana 1
columnas_ya_generadas = list(Xs[1].columns.values)

---

# Creación de features

## Cantidad de apariciones en subastas dentro de la ventana

In [7]:
def cantidad_apariciones_en_ventana(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : "count"}).rename(columns={"date": "cantidad_apariciones_en_ventana_auctions"})

In [8]:
features["cantidad_apariciones_en_ventana_auctions"] = (cantidad_apariciones_en_ventana, ["cantidad_apariciones_en_ventana_auctions"])

---

## Cantidad de apariciones discriminando períodos

## Cantidad de apariciones por período horario

In [9]:
def cantidad_apariciones_por_periodo_horario(dataframe, nro_ventana):
    features = dataframe.groupby("device_id").agg( \
       {"hora_2_a_6" : "sum",
        "hora_6_a_11" : "sum",
        "hora_11_a_14" : "sum",
        "hora_14_a_18" : "sum",
        "hora_18_a_23" : "sum",
        "hora_23_a_2" : "sum"})
    features.columns = ["apariciones_" + x + "_auctions" for x in features.columns]
    return features

In [10]:
columnas_generadas = ["apariciones_" + x + "_auctions" for x in ["hora_2_a_6", "hora_6_a_11", "hora_11_a_14", "hora_14_a_18", "hora_18_a_23", "hora_23_a_2"]]
features["cantidad_apariciones_por_periodo_horario"] = (cantidad_apariciones_por_periodo_horario, columnas_generadas)

# Cantidad de apariciones según en qué dia de la ventana se encuentra

In [11]:
def cantidad_apariciones_por_dia(dataframe, nro_ventana):
    apariciones_por_dia = dataframe.groupby(["device_id", "dia"]).agg(({"date" : "count"}))
    apariciones_por_dia = apariciones_por_dia.unstack().droplevel(level=0, axis=1)
    cantidad_de_dias = len(apariciones_por_dia.columns)
    nuevas_columnas = []
    for dia in range(cantidad_de_dias):
        nuevas_columnas.append(dia+1)
    apariciones_por_dia.columns = nuevas_columnas
    for dia in range(1, 4):
        if dia not in list(apariciones_por_dia.columns.values):
            apariciones_por_dia.insert(dia, dia, 0)
    apariciones_por_dia.columns = ["apariciones_en_dia_{}_auctions".format(dia) for dia in range(1,4)]
    return apariciones_por_dia

In [12]:
columnas_generadas = ["apariciones_en_dia_{}_auctions".format(dia) for dia in range(1,4)]
features["cantidad_apariciones_por_dia"] = (cantidad_apariciones_por_dia, columnas_generadas)

---

## Sistema operativo (Android / iOS)

In [13]:
def cantidad_so_registrados(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"ref_type_id" : "nunique"}).rename(columns={"ref_type_id": "cantidad_so_registrados_auctions"})

In [14]:
features["cantidad_so_registrados"] = (cantidad_so_registrados, ["cantidad_so_registrados_auctions"])

In [15]:
def sistema_operativo_principal(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"ref_type_id" : (lambda x: round(x.mean(), 0))}).rename(columns={"ref_type_id": "so_mas_usado_auctions"})

In [16]:
features["sistema_operativo_principal"] = (sistema_operativo_principal, ["so_mas_usado_auctions"])

---

## Tiempo medio entre apariciones

In [17]:
def tiempo_medio_entre_apariciones(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : (lambda x: (x.sub(x.shift()).mean().total_seconds()))}).rename(columns={"date": "tiempo_medio_entre_apariciones_auctions"})

In [18]:
features["tiempo_medio_entre_apariciones"] = (tiempo_medio_entre_apariciones, ["tiempo_medio_entre_apariciones_auctions"])

---

## Tiempo máximo entre apariciones

In [19]:
def tiempo_maximo_entre_apariciones(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : (lambda x: (x.sub(x.shift()).max().total_seconds()))}).rename(columns={"date": "tiempo_maximo_entre_apariciones_auctions"})

In [20]:
features["tiempo_maximo_entre_apariciones"] = (tiempo_maximo_entre_apariciones, ["tiempo_maximo_entre_apariciones_auctions"])

---

## Tiempo mínimo entre apariciones

In [21]:
def tiempo_minimo_entre_apariciones(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : (lambda x: (x.sub(x.shift()).min().total_seconds()))}).rename(columns={"date": "tiempo_minimo_entre_apariciones_auctions"})

In [22]:
features["tiempo_minimo_entre_apariciones"] = (tiempo_minimo_entre_apariciones, ["tiempo_minimo_entre_apariciones_auctions"])

---

## Cantidad de apariciones por fuente

In [23]:
def cantidad_apariciones_segun_source(dataframe, nro_ventana):
    feature = dataframe.groupby(["device_id", "source_id"])["date"].agg("count").to_frame().unstack().droplevel(level=0, axis=1)
    for source in range(0, 10):
        if source not in list(feature.columns.values):
            feature.insert(source, source, 0)
    feature.columns = ["apariciones_en_source_{}_auctions".format(x) for x in range(10)]
    return feature

In [24]:
columnas_generadas = ["apariciones_en_source_{}_auctions".format(x) for x in range(10)]
features["cantidad_apariciones_segun_source"] = (cantidad_apariciones_segun_source, columnas_generadas)

---

## Tiempo desde la última aparición hasta el fin de la ventana

In [25]:
def tiempo_desde_ult_aparicion_hasta_fin_ventana(dataframe, nro_ventana):
    return (np.datetime64("2019-04-2{}".format(nro_ventana)) - dataframe.groupby("device_id").agg({"date" : "max"})["date"]).dt.total_seconds().to_frame().rename(columns={"date": "tiempo_ultima_aparicion_hasta_fin_ventana_auctions"})

In [26]:
features["tiempo_desde_ult_aparicion_hasta_fin_ventana"] = (tiempo_desde_ult_aparicion_hasta_fin_ventana, ["tiempo_ultima_aparicion_hasta_fin_ventana_auctions"])

---

## Tiempo desde la primera aparición hasta el fin de la ventana

In [27]:
def tiempo_desde_prim_aparicion_hasta_fin_ventana(dataframe, nro_ventana):
    return (np.datetime64("2019-04-2{}".format(nro_ventana)) - dataframe.groupby("device_id").agg({"date" : "min"})["date"]).dt.total_seconds().to_frame().rename(columns={"date": "tiempo_primera_aparicion_hasta_fin_ventana_auctions"})

In [28]:
features["tiempo_desde_prim_aparicion_hasta_fin_ventana"] = (tiempo_desde_prim_aparicion_hasta_fin_ventana, ["tiempo_primera_aparicion_hasta_fin_ventana_auctions"])

---

In [29]:
def tiempo_medio_hasta_fin(grupo, nro_ventana):
    fecha_fin_ventana = np.datetime64("2019-04-2{}".format(nro_ventana))
    fecha_minima = grupo["date"].min()
    delta = (grupo["date"] - fecha_minima).mean()
    return delta.total_seconds()

def tiempo_desde_aparicion_media_hasta_fin_ventana(dataframe, nro_ventana):
    return dataframe[["device_id", "date"]].groupby("device_id").agg(lambda x: tiempo_medio_hasta_fin(x, nro_ventana)) \
        .rename(columns={"date" : "tiempo_aparicion_media_hasta_fin_ventana_auctions"})


In [30]:
features["tiempo_desde_aparicion_media_hasta_fin_ventana"] = (tiempo_desde_aparicion_media_hasta_fin_ventana, ["tiempo_aparicion_media_hasta_fin_ventana_auctions"])

---

# Genero los features

## Función generadora de features en ventanas

In [31]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos, ventana_inicial, ventana_final):
    """El rango se toma como python, [ventana_inicial, ventana_final)"""
    for ventana_nro in range(ventana_inicial, ventana_final):
        feature = generador_feature(dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]], ventana_nro)
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True, how="left")

### Genero los features

In [34]:
COLUMNAS_QUE_GENERA_EL_FEATURE = 1
FUNCION_PARA_CALCULAR_EL_FEATURE = 0

for nombre_feature in features:    
    columnas_generadas = features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE]
    feature_ya_generado = False
    for columna in columnas_generadas:
        if columna in columnas_ya_generadas:
            feature_ya_generado = True
            break
    
    if feature_ya_generado:
        continue

    generar_feature_en_ventanas(auctions, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs, 1, 5)
    generar_feature_en_ventanas(auctions_competencia, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs_predecir, 7, 8)    
    columnas_ya_generadas.extend(features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE])

---

# Guardado de los sets de entrenamiento y el set de prediccion

In [ ]:
for ventana_nro in range(1, 5):
    Xs[ventana_nro].to_csv("../../features/ventana_{}_entrenar_auctions.csv".format(ventana_nro))

Xs_predecir[7].to_csv("../../features/predecir_auctions.csv")

---